# a3za

### STATUS:
- **PRAISE**: finally got importing vgg16 to work great (PTL, seriously!!)
 - https://github.com/pierluigiferrari/fcn8s_tensorflow#download-pre-trained-vgg-16
- **ISSUE**: when I try to see how many examples are in my train_dataset, there seems to be way more than expected (and possibly repeating forever)
 - **SOLVED**: iterator.get_next() can't go in a while loop; it needs to be part of the graph
- **ISSUE**: I don't think the iterator + feed_dict is the correct combination; I'd like to put the iterator in the graph and not feed, but need to figure out how to set values that are part of the vgg16 graph
- **ISSUE**: it would be nice to see the vgg16 graph in TensorBoard (well, the whole thing in TensorBoard, really)

### INFO:
- Train dataset has 91154 examples
- Val dataset has 12750 examples
- Test dataset has 24378 examples

In [1]:
import os
import numpy as np
import tensorflow as wtf
import data_utils as du
from fcn8 import FCN8s
from helpers.visualization_utils import print_segmentation_onto_image, create_video_from_images
from math import ceil

/home/noodles/.conda/envs/turntable/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
BATCH_SIZE = 16
EPOCHS = 1

num_train, num_val, num_test = 70, 10, 20
train_data,val_data,test_data = du.create_dicts(num_train, num_val, num_test)

train_dataset = du.create_dataset_fcn8('train', train_data)
train_dataset = train_dataset.shuffle(2000)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.repeat(EPOCHS)
train_iterator = train_dataset.make_one_shot_iterator()

val_dataset = du.create_dataset_fcn8('val', val_data)
val_dataset = val_dataset.batch(BATCH_SIZE)
val_iterator = val_dataset.make_one_shot_iterator()

In [3]:
model = FCN8s(model_load_dir=None,
              tags=None,
              vgg16_dir='vgg16',
              num_classes=1,
#               variables_load_dir='vgg16/variables',
              variables_load_dir=None,
              train_iterator=train_iterator,
              val_iterator=val_iterator)

TensorFlow Version: 1.7.0
INFO:tensorflow:Restoring parameters from b'vgg16/variables/variables'
Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
epochs = 1
num_train_images = 91154
num_val_images = 12750

def learning_rate_schedule(step):
    if step <= 10000: return 0.00001
    elif 10000 < step <= 20000: return 0.000001
    elif 20000 < step <= 40000: return 0.0000003
    else: return 0.0000001

model.train(epochs=epochs,
            steps_per_epoch=ceil(num_train_images/BATCH_SIZE),
            learning_rate_schedule=learning_rate_schedule,
            keep_prob=0.5,
            l2_regularization=0.0,
#             eval_dataset='val',
            eval_frequency=2,
            val_steps=ceil(num_val_images/BATCH_SIZE),
            metrics={'loss', 'mean_iou', 'accuracy'},
            save_during_training=True,
            save_dir='a3za_model',
            save_best_only=True,
            save_tags=['default'],
            save_name='(batch-size-16)',
            save_frequency=2,
            saver='saved_model',
            monitor='loss',
            record_summaries=True,
            summaries_frequency=10,
            summaries_dir='tensorboard_log/a3za',
            summaries_name='configuration_01',
            training_loss_display_averaging=3)

Default GPU Device: /device:GPU:0
Epoch 1/1:  24%|██▍       | 1383/5698 [33:35<1:44:47,  1.46s/it, loss=9.69e-07, learning rate=1e-05]

KeyboardInterrupt: 

## Number of Examples
Used to count number of examples in train, val, and test datasets. Set "mode" and run.

In [ ]:
wtf.reset_default_graph()

EPOCHS = 1
BATCH_SIZE = 1

num_train, num_val, num_test = 70, 10, 20
train_data,val_data,test_data = du.create_dicts(num_train, num_val, num_test)

mode = 'val'
if mode=='train':
    patients = train_data
elif mode=='val':
    patients = val_data
elif mode=='test':
    patients = test_data

dataset = du.create_dataset_fcn8(mode, patients)
dataset = dataset.batch(3000)
dataset = dataset.repeat(EPOCHS)
iterator = dataset.make_one_shot_iterator()
x, y = iterator.get_next()

with wtf.Session() as sess:
    num_ex = 0
    try:
        while True:
            y_np = sess.run(y)
            num_ex += len(y_np)
            print("{}".format(num_ex))
    except wtf.errors.OutOfRangeError:
        print("{}".format(num_ex))

## FCN8 Implementation

In [ ]:
wtf.reset_default_graph()

EPOCHS = 1
BATCH_SIZE = 50

num_train, num_val, num_test = 70, 10, 20
train_data,val_data,test_data = du.create_dicts(num_train, num_val, num_test)

train_dataset = du.create_dataset_fcn8('train', train_data)
train_dataset = train_dataset.shuffle(2000)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.repeat(EPOCHS)
iterator = train_dataset.make_one_shot_iterator()

In [ ]:
VGG_PATH = 'vgg16'
IMAGE_SHAPE = (192, 224)
DROPOUT = 0.75

X_batch, gt_batch = iterator.get_next()
correct_label = wtf.placeholder(wtf.float32, [None, IMAGE_SHAPE[0], IMAGE_SHAPE[1], 1])
#correct_label = gt_batch
learning_rate = wtf.placeholder(wtf.float32)
keep_prob = wtf.placeholder(wtf.float32)

def load_vgg(sess, vgg_path):
    model = wtf.saved_model.loader.load(sess, ['vgg16'], vgg_path)
    graph = wtf.get_default_graph()
    image_input = graph.get_tensor_by_name('image_input:0')
    #image_input = X_batch
    print(image_input)
    keep_prob = graph.get_tensor_by_name('keep_prob:0')
    layer3 = graph.get_tensor_by_name('layer3_out:0')
    layer4 = graph.get_tensor_by_name('layer4_out:0')
    layer7 = graph.get_tensor_by_name('layer7_out:0')
    return image_input, keep_prob, layer3, layer4, layer7

def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    # Use a shorter variable name for simplicity
    layer3, layer4, layer7 = vgg_layer3_out, vgg_layer4_out, vgg_layer7_out
    # Apply 1x1 convolution in place of fully connected layer
    fcn8 = wtf.layers.conv2d(layer7, filters=num_classes, kernel_size=1, name="fcn8")
    # Upsample fcn8 with size depth=(4096?) to match size of layer 4 so that we can add skip connection with 4th layer
    fcn9 = wtf.layers.conv2d_transpose(fcn8, filters=layer4.get_shape().as_list()[-1],
                                       kernel_size=4, strides=(2, 2), padding='SAME', name="fcn9")
    # Add a skip connection between current final layer fcn8 and 4th layer
    fcn9_skip_connected = wtf.add(fcn9, layer4, name="fcn9_plus_vgg_layer4")
    # Upsample again
    fcn10 = wtf.layers.conv2d_transpose(fcn9_skip_connected, filters=layer3.get_shape().as_list()[-1],
                                        kernel_size=4, strides=(2, 2), padding='SAME', name="fcn10_conv2d")
    # Add skip connection
    fcn10_skip_connected = wtf.add(fcn10, layer3, name="fcn10_plus_vgg_layer3")
    # Upsample again
    fcn11 = wtf.layers.conv2d_transpose(fcn10_skip_connected, filters=num_classes,
                                        kernel_size=16, strides=(8, 8), padding='SAME', name="fcn11")
    return fcn11

def optimize_fn(nn_last_layer, correct_label, learning_rate, num_classes):
    # Reshape 4D tensors to 2D, each row represents a pixel, each column a class
    logits = wtf.reshape(nn_last_layer, (-1, num_classes), name="fcn_logits")
    correct_label_reshaped = wtf.reshape(correct_label, (-1, num_classes))
    # Calculate distance from actual labels using cross entropy
    cross_entropy = wtf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=correct_label_reshaped[:])
    # Take mean for total loss
    loss_op = wtf.reduce_mean(cross_entropy, name="fcn_loss")
    # The model implements this operation to find the weights/parameters that would yield correct pixel labels
    train_op = wtf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op, name="fcn_train_op")
    return logits, train_op, loss_op

def train_nn(sess, epochs, batch_size, train_op,
             cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    keep_prob_value = 0.5
    
    learning_rate_value = 0.001
    for epoch in range(epochs):
        total_loss = 0
        try:
            while True:
                X_batch_np, gt_batch_np = sess.run([X_batch, gt_batch])
#                 X_batch, gt_batch = iterator.get_next()
                loss, _ = sess.run([cross_entropy_loss, train_op],
                                   feed_dict={input_image: X_batch_np, correct_label: gt_batch_np,
                                              keep_prob: keep_prob_value, learning_rate: learning_rate_value})
#                 loss, _ = sess.run([cross_entropy_loss, train_op],
#                                    feed_dict={keep_prob: keep_prob_value, learning_rate:learning_rate_value})
                total_loss += loss
                print("Sanity check {}".format(total_loss))
        except wtf.errors.OutOfRangeError:
            pass
        print("EPOCH {} ...".format(epoch + 1))
        print("Loss = {:.3f}".format(total_loss))
        print()

In [ ]:
def run():
    with wtf.Session() as sess:
        image_input, keep_prob, layer3, layer4, layer7 = load_vgg(sess, VGG_PATH)
        model_output = layers(layer3, layer4, layer7, 1)
        logits, train_op, cross_entropy_loss = optimize_fn(model_output, correct_label, learning_rate, 1)
        sess.run(wtf.global_variables_initializer())
        sess.run(wtf.local_variables_initializer())
        
        train_nn(sess, EPOCHS, BATCH_SIZE,
                 train_op, cross_entropy_loss, image_input,
                 correct_label, keep_prob, learning_rate)

In [ ]:
if __name__ == '__main__':
    run()